In [1]:
import csv
import pandas as pd
import os
from tqdm import tqdm
import time
import numpy as np
import cv2

In [2]:

%matplotlib inline
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-trainval', dataroot='dataset/nuscenes/trian', verbose=True)
nusc_test = NuScenes(version='v1.0-test', dataroot='dataset/nuscenes/test', verbose=True)



Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 32.054 seconds.
Reverse indexing ...
Done reverse indexing in 8.2 seconds.
Loading NuScenes tables for version v1.0-test...
23 category,
8 attribute,
4 visibility,
0 instance,
12 sensor,
1800 calibrated_sensor,
462901 ego_pose,
15 log,
150 scene,
6008 sample,
462901 sample_data,
0 sample_annotation,
4 map,
Done loading in 3.155 seconds.
Reverse indexing ...
Done reverse indexing in 0.9 seconds.


In [3]:
length01 = len(nusc.scene)
# length01 = 2
sensor1 = 'CAM_FRONT'
sensor2 = 'CAM_BACK'
sensor3 = 'CAM_BACK_LEFT'
sensor4 = 'CAM_BACK_RIGHT'
sensor5 = 'CAM_FRONT_RIGHT'
sensor6 = 'CAM_FRONT_LEFT'
csv_data_singlecamera = [] # store the data for training, only one camera is used
csv_data_allcamera = [] # store data from all cameras
filepath = 'dataset/nuscenes/trian/'

# extract img path and the xy coordinate of samples in every scene
for i in tqdm(range(length01), desc="processing of scenes"):

    time.sleep(0.001)
    my_scene = nusc.scene[i]
    num_sample = my_scene['nbr_samples']
    sample_token = None
    sample_token_next = None
    csv_data_scene = [] # store data of single scene

    # extract data from each sample
    for j in range(num_sample):
        if j == 0:
            sample_token = my_scene['first_sample_token']
            my_sample = nusc.get('sample', sample_token)
            sample_token_next = my_sample['next']
        elif j == (num_sample - 1):
            sample_token = my_scene['last_sample_token']
            my_sample = nusc.get('sample', sample_token)
        else:
            sample_token = sample_token_next
            my_sample = nusc.get('sample', sample_token)
            sample_token_next = my_sample['next']
        
        # get the data of different sensor (camera in different positions)
        cam_data_01 = nusc.get('sample_data', my_sample['data'][sensor1]) # 'CAM_FRONT'
        cam_data_02 = nusc.get('sample_data', my_sample['data'][sensor2]) # 'CAM_BACK'
        cam_data_03 = nusc.get('sample_data', my_sample['data'][sensor3]) # 'CAM_BACK_LEFT'
        cam_data_04 = nusc.get('sample_data', my_sample['data'][sensor4]) # 'CAM_BACK_RIGHT'
        cam_data_05 = nusc.get('sample_data', my_sample['data'][sensor5]) # 'CAM_FRONT_RIGHT'
        cam_data_06 = nusc.get('sample_data', my_sample['data'][sensor6]) # 'CAM_FRONT_LEFT'
        img_xy = []
        img_xy.append(cam_data_01['filename'])
        img_xy.append(cam_data_02['filename'])
        img_xy.append(cam_data_03['filename'])
        img_xy.append(cam_data_04['filename'])
        img_xy.append(cam_data_05['filename'])
        img_xy.append(cam_data_06['filename'])
        ego_pose_token = cam_data_01['ego_pose_token']
        ego_pose = nusc.get('ego_pose', ego_pose_token)['translation']
        img_xy.append(ego_pose[0])
        img_xy.append(ego_pose[1])

        # size of img_xy: 8*1 (6 img path + x and y coordinate), it collects all needed data in one sample
        csv_data_scene.append(img_xy) # shape: num_sample * 8

    # print(np.shape(csv_data_scene))

    # deal with all extracted data in one scene
    # 4 imges and their corresponding xy coordinates for input and 12 xy coordinates as prediction
    num_group = (num_sample - 16) // 2 + 1
    
    for t in range(num_group):
        
        m = t * 2
        data_group_onecamera = []
        data_group_allcamera = []
        
        # for each group for single camera, data is (img path * 1 + x + y) * 4 + (x + y) * 12
        # for each group for all cameras, data id (img path * 6 + x + y) * 4 + (x + y) * 12
        for n in range(16):
            if n < 4:
                # data_group_onecamera += csv_data_scene[m + n][0] + csv_data_scene[m + n][-2:]
                data_group_onecamera.append(csv_data_scene[m + n][0])
                data_group_onecamera += csv_data_scene[m + n][-2:]
                data_group_allcamera += csv_data_scene[m + n][0:]
            else:
                data_group_onecamera += csv_data_scene[m + n][-2:]
                data_group_allcamera += csv_data_scene[m + n][-2:]
        
        # print('data_group_onecamera', np.shape(data_group_onecamera))

        csv_data_singlecamera.append(data_group_onecamera)
        csv_data_allcamera.append(data_group_allcamera)

# create csv files to store the data from one camera and data from all cameras

# CSV file path
csv_file1 = "dataset/nuscenes/trian/data_one_camera.csv"
csv_file2 = "dataset/nuscenes/trian/data_all_camera.csv"

# write the data in csv files
with open(csv_file1, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data_singlecamera)

with open(csv_file2, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data_allcamera)

print(f"Data has been written to {csv_file1}")
print(f"Data has been written to {csv_file2}")

l1 = np.array(csv_data_singlecamera)
l2 = np.array(csv_data_allcamera)
print('l1', l1.shape, 'l2', l2.shape)
        

processing of scenes: 100%|██████████| 850/850 [00:01<00:00, 466.42it/s]


Data has been written to dataset/nuscenes/trian/data_one_camera.csv
Data has been written to dataset/nuscenes/trian/data_all_camera.csv
l1 (10954, 36) l2 (10954, 56)


In [4]:
length01 = len(nusc_test.scene)
# length01 = 2
sensor1 = 'CAM_FRONT'
sensor2 = 'CAM_BACK'
sensor3 = 'CAM_BACK_LEFT'
sensor4 = 'CAM_BACK_RIGHT'
sensor5 = 'CAM_FRONT_RIGHT'
sensor6 = 'CAM_FRONT_LEFT'
csv_data_singlecamera = [] # store the data for training, only one camera is used
csv_data_allcamera = [] # store data from all cameras

# extract img path and the xy coordinate of samples in every scene
for i in tqdm(range(length01), desc="processing of scenes"):

    time.sleep(0.001)
    my_scene = nusc_test.scene[i]
    num_sample = my_scene['nbr_samples']
    sample_token = None
    sample_token_next = None
    csv_data_scene = [] # store data of single scene

    # extract data from each sample
    for j in range(num_sample):
        if j == 0:
            sample_token = my_scene['first_sample_token']
            my_sample = nusc_test.get('sample', sample_token)
            sample_token_next = my_sample['next']
        elif j == (num_sample - 1):
            sample_token = my_scene['last_sample_token']
            my_sample = nusc_test.get('sample', sample_token)
        else:
            sample_token = sample_token_next
            my_sample = nusc_test.get('sample', sample_token)
            sample_token_next = my_sample['next']
        
        # get the data of different sensor (camera in different positions)
        cam_data_01 = nusc_test.get('sample_data', my_sample['data'][sensor1]) # 'CAM_FRONT'
        cam_data_02 = nusc_test.get('sample_data', my_sample['data'][sensor2]) # 'CAM_BACK'
        cam_data_03 = nusc_test.get('sample_data', my_sample['data'][sensor3]) # 'CAM_BACK_LEFT'
        cam_data_04 = nusc_test.get('sample_data', my_sample['data'][sensor4]) # 'CAM_BACK_RIGHT'
        cam_data_05 = nusc_test.get('sample_data', my_sample['data'][sensor5]) # 'CAM_FRONT_RIGHT'
        cam_data_06 = nusc_test.get('sample_data', my_sample['data'][sensor6]) # 'CAM_FRONT_LEFT'
        img_xy = []
        img_xy.append(cam_data_01['filename'])
        img_xy.append(cam_data_02['filename'])
        img_xy.append(cam_data_03['filename'])
        img_xy.append(cam_data_04['filename'])
        img_xy.append(cam_data_05['filename'])
        img_xy.append(cam_data_06['filename'])
        ego_pose_token = cam_data_01['ego_pose_token']
        ego_pose = nusc_test.get('ego_pose', ego_pose_token)['translation']
        img_xy.append(ego_pose[0])
        img_xy.append(ego_pose[1])

        # size of img_xy: 8*1 (6 img path + x and y coordinate), it collects all needed data in one sample
        csv_data_scene.append(img_xy)

    # deal with all extracted data in one scene
    # 4 imges and their corresponding xy coordinates for input and 12 xy coordinates as prediction
    num_group = (num_sample - 16) // 2 + 1
    
    for t in range(num_group):
        m = t * 2
        data_group_onecamera = []
        data_group_allcamera = []
        
        # for each group for single camera, data is (img path * 1 + x + y) * 4 + (x + y) * 12
        # for each group for all cameras, data id (img path * 6 + x + y) * 4 + (x + y) * 12
        for n in range(16):
            if n < 4:
                # data_group_onecamera += csv_data_scene[m + n][0] + csv_data_scene[m + n][-2:]
                data_group_onecamera.append(csv_data_scene[m + n][0])
                data_group_onecamera += csv_data_scene[m + n][-2:]
                data_group_allcamera += csv_data_scene[m + n][0:]
            else:
                data_group_onecamera += csv_data_scene[m + n][-2:]
                data_group_allcamera += csv_data_scene[m + n][-2:]

        csv_data_singlecamera.append(data_group_onecamera)
        csv_data_allcamera.append(data_group_allcamera)

# create csv files to store the data from one camera and data from all cameras

# CSV file path
csv_file1 = "dataset/nuscenes/test/data_one_camera_test.csv"
csv_file2 = "dataset/nuscenes/test/data_all_camera_test.csv"

# write the data in csv files
with open(csv_file1, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data_singlecamera)

with open(csv_file2, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data_allcamera)

print(f"Data has been written to {csv_file1}")
print(f"Data has been written to {csv_file2}")

l1 = np.array(csv_data_singlecamera)
l2 = np.array(csv_data_allcamera)
print('l1', l1.shape, 'l2', l2.shape)
        

processing of scenes: 100%|██████████| 150/150 [00:00<00:00, 491.13it/s]


Data has been written to dataset/nuscenes/test/data_one_camera_test.csv
Data has been written to dataset/nuscenes/test/data_all_camera_test.csv
l1 (1934, 36) l2 (1934, 56)


In [25]:
example_scene = nusc.scene[0]
example_token = example_scene['first_sample_token']
example_sample = nusc.get('sample', example_token)
cam_data = nusc.get('sample_data', example_sample['data']['CAM_FRONT'])
cam_data2 = nusc.get('sample_data', example_sample['data']['CAM_BACK_RIGHT'])
imgpath = []
imgpath.append('dataset/nuscenes/trian/' + cam_data['filename'])
ego_pose_token1 = cam_data['ego_pose_token']
ego_pose = nusc.get('ego_pose', ego_pose_token1)['translation']
ego_pose_token2 = cam_data2['ego_pose_token']
ego_pose2 = nusc.get('ego_pose', ego_pose_token2)['translation']
imgpath.append(ego_pose[0])
imgpath.append(ego_pose[1])
imgpath.append(ego_pose2[0])
imgpath.append(ego_pose2[1])
imgpath

['dataset/nuscenes/trian/samples/CAM_FRONT/n015-2018-07-18-11-07-57+0800__CAM_FRONT__1531883530412470.jpg',
 1010.1102882349232,
 610.6567106479714,
 1010.1196021097104,
 610.7194357591594]

In [23]:
img = cv2.imread(imgpath[0])
cv2.imshow('Image' ,img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
import csv

# 数据
data = [
    ["Name", "Age", "City"],
    ["Alice", 25, "New York"],
    ["Bob", 30, "Los Angeles"],
    ["Charlie", 22, "Chicago"]
]

# CSV文件路径
csv_file = "example.csv"

# 写入数据到CSV文件
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"Data has been written to {csv_file}")


Data has been written to example.csv


In [6]:
import csv

# CSV文件路径
csv_file = "example.csv"  # 你需要将文件路径替换为你的CSV文件路径

# 创建一个空列表，用于存储每一行的数据
data_list = []

# 逐行读取CSV文件并存储为列表
with open(csv_file, mode='r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        data_list.append(row)

# 打印每一行的数据
for row in data_list:
    print(row)
np.shape(data_list)

['Name', 'Age', 'City']
['Alice', '25', 'New York']
['Bob', '30', 'Los Angeles']
['Charlie', '22', 'Chicago']


(4, 3)

In [10]:
data_list[0][0]

'Name'

In [6]:
sensor = 'CAM_FRONT'
cam_front_data_01 = nusc.get('sample_data', my_sample['data'][sensor])
cam_front_data_02 = nusc.get('sample_data', my_sample_02['data'][sensor])
cam_front_data_01


{'token': 'e3d495d4ac534d54b321f50006683844',
 'sample_token': 'ca9a282c9e77460f8360f564131a8af5',
 'ego_pose_token': 'e3d495d4ac534d54b321f50006683844',
 'calibrated_sensor_token': '1d31c729b073425e8e0202c5c6e66ee1',
 'timestamp': 1532402927612460,
 'fileformat': 'jpg',
 'is_key_frame': True,
 'height': 900,
 'width': 1600,
 'filename': 'samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402927612460.jpg',
 'prev': '',
 'next': '68e8e98cf7b0487baa139df808641db7',
 'sensor_modality': 'camera',
 'channel': 'CAM_FRONT'}

In [7]:
cam_front_data_02

{'token': '4b6870ae200c4b969b91c50a9737f712',
 'sample_token': '39586f9d59004284a7114a68825e8eec',
 'ego_pose_token': '4b6870ae200c4b969b91c50a9737f712',
 'calibrated_sensor_token': '1d31c729b073425e8e0202c5c6e66ee1',
 'timestamp': 1532402928112460,
 'fileformat': 'jpg',
 'is_key_frame': True,
 'height': 900,
 'width': 1600,
 'filename': 'samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402928112460.jpg',
 'prev': 'e92333479b9048c88c5db855621f4401',
 'next': 'cb21f6eaada04b42a1b05d2c7a34e857',
 'sensor_modality': 'camera',
 'channel': 'CAM_FRONT'}

In [8]:
img_path = []
ego_pose = []
img_path.append(cam_front_data_01['filename'])
img_path.append(cam_front_data_02['filename'])
ego_pose.append(cam_front_data_01['ego_pose_token'])
ego_pose.append(cam_front_data_02['ego_pose_token'])

In [9]:
import cv2
path = 'v1.0-mini/' + img_path[0]
print(path)
img = cv2.imread(path)
cv2.imshow('Image' ,img)
cv2.waitKey(0)
cv2.destroyAllWindows()


v1.0-mini/samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402927612460.jpg


In [13]:
pose01 = nusc.get('ego_pose', ego_pose[0])['translation']
pose01

[411.4199861830012, 1181.197175631848, 0.0]

In [11]:
ego_pose

['e3d495d4ac534d54b321f50006683844', '4b6870ae200c4b969b91c50a9737f712']

In [13]:
a = [1,2,3,4,5]
b = ['ichbindeinvater', 1, 2]
c = []
c += a + b

c

[1, 2, 3, 4, 5, 'ichbindeinvater', 1, 2]